In [1]:
# Importación de librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Carga de datos desde un archivo CSV
df = pd.read_csv('../generate_test_data/answers.csv')
df

In [3]:
# Creación de la variable objetivo (y) como promedio de algunas columnas
y = (df['In02'] + df['In03'] + df['In04'] + df['In05'])/20

In [ ]:
# Definición de las características (X) para el modelo
headers = ['Ac02','Ac03','Ac06','Ac10','Ac13','Ec01','Ec03','Ec05','Fa02','Fa03','Fa07','Fa08','Fa09','Mo02','Mo08','Mo09','Mo10','Mo11','Sa01','Sa02','Sa03','Sa04','So01','So04','So07','So08']
X = df[headers]
X.head()

## Regresion Lineal

In [5]:
# Dividir los datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Regresión Lineal
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()

linear_model.fit(X_train, y_train)

In [ ]:
# Evaluación del modelo de regresión lineal
from sklearn.metrics import mean_squared_error, r2_score

y_pred = linear_model.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

In [ ]:
# Análisis estadístico más detallado con statsmodels
import statsmodels.api as sm

X_train_sm = sm.add_constant(X_train)


sm_model = sm.OLS(y_train, X_train_sm).fit()


print(sm_model.summary())

## Regression Tree Classifier

In [ ]:
# Árbol de Regresión
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=4, random_state=42)
tree_reg.fit(X_train, y_train)

In [10]:
# Exportar el árbol de decisión para visualización
from sklearn.tree import export_graphviz
export_graphviz(
tree_reg,
out_file='desertion_regression.dot',
feature_names=headers,
rounded=True,
filled=True
)

In [ ]:
# Visualizar el árbol de decisión (requiere graphviz instalado)
from graphviz import Source
Source.from_file('desertion_regression.dot')

In [ ]:
# Evaluación del árbol de regresión
from sklearn.metrics import mean_squared_error
import numpy as np

y_pred = tree_reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Error cuadratico medio (MSE): {mse:.2f}")
print(f"Raiz del error Cuadratico Medio (RMSE): {rmse:.2f}")

In [ ]:
# Cálculo del Error Medio Absoluto
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)
print(f"Error Medio Absoluto (MAE): {mae:.2f}")

In [ ]:
# Comparación del rendimiento en entrenamiento vs prueba
y_train_pred = tree_reg.predict(X_train)

rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(f"Training RMSE: {rmse_train:.2f}")

rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Testing RMSE: {rmse_test:.2f}")

In [ ]:
# Visualización de la importancia de las características
importances = tree_reg.feature_importances_

feature_importances = pd.Series(importances, index=X.columns)
feature_importances.sort_values(ascending=False).plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Importancia de las Variables')

plt.savefig('importances.png', format='png', bbox_inches='tight') 

plt.show() 

In [16]:
# Guardar las predicciones en un archivo CSV
predictions_df = pd.DataFrame({
    'Actual': y_test,
    'Previsto': y_pred
})
predictions_df.to_csv('../predicciones.csv', index=False)
